In [1]:
import json
import re
import pandas as pd
import numpy as np

In [2]:
f = open('nobackup/photos.json', 'r')
meta = []
for line in f:
    meta.append(json.loads(line))
f.close
meta = pd.DataFrame(meta)
meta.head   

<bound method NDFrame.head of                       photo_id             business_id  \
0       zsvj7vloL4L5jhYyPIuVwg  Nk-SJhPlDBkAZvfsADtccA   
1       HCUdRJHHm_e0OCTlZetGLg  yVZtL5MmrpiivyCIrVkGgA   
2       vkr8T0scuJmGVvN2HJelEA  _ab50qdWOk0DdB6XOrBitw   
3       pve7D6NUrafHW3EAORubyw  SZU9c8V2GuREDN5KgyHFJw   
4       H52Er-uBg6rNrHcReWTD2w  Gzur0f0XMkrVxIwYJvOt2g   
...                        ...                     ...   
200095  4Zia9NkAfQNjMfcIDhwJ-g  2HxkdqHmbYGj_BH1bLaiSw   
200096  KB96KRZRhRm8hUkI-OpGEA  _gVyuTRb_6HM-SNtqbpevQ   
200097  Klmojvaf2_2dP1XKzTsFmQ  NUyEOjfAl3HvkpzSpdwqeA   
200098  FNEiq7Mogec7t31OaU5juw  hE6YsHHV0fCz_UrGS4o6VA   
200099  NHEtLh7APk7Yssjo0h45VA  VIYvcX9SScnqmoI0so1KZA   

                                                  caption    label  
0       Nice rock artwork everywhere and craploads of ...   inside  
1                                                          outside  
2                                          oyster shooter    drink

In [3]:
meta.label.unique()

array(['inside', 'outside', 'drink', 'food', 'menu'], dtype=object)

##### Save list of non-food item names

In [4]:
non_food = meta[meta.label.isin(['inside', 'outside', 'drink', 'menu'])]
non_food.photo_id = [id + '.jpg' for id in non_food.photo_id]
non_food.shape

/u/x/i/xianshi/miniconda3/envs/bert_hw/lib/python3.7/site-packages/pandas/core/generic.py:5516: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


(91948, 4)

In [5]:
non_food_list = ' '.join(non_food.photo_id)
f = open('non_food_list.txt', 'w')
f.write(non_food_list)
f.close()

##### Select and save (not in food_101) captioned food item list

In [14]:
food = meta[meta.label == 'food']
food.loc[:,'caption'] = [c.strip().lower() for c in food.caption]

/u/x/i/xianshi/miniconda3/envs/bert_hw/lib/python3.7/site-packages/pandas/core/indexing.py:1951: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[selected_item_labels] = value


In [24]:
f = open('nobackup/food101/food-101/meta/labels.txt', 'r')
label_list = f.readlines()
f.close()

In [25]:
label_list = [l.strip().lower() for l in label_list]
label_list

['apple pie',
 'baby back ribs',
 'baklava',
 'beef carpaccio',
 'beef tartare',
 'beet salad',
 'beignets',
 'bibimbap',
 'bread pudding',
 'breakfast burrito',
 'bruschetta',
 'caesar salad',
 'cannoli',
 'caprese salad',
 'carrot cake',
 'ceviche',
 'cheesecake',
 'cheese plate',
 'chicken curry',
 'chicken quesadilla',
 'chicken wings',
 'chocolate cake',
 'chocolate mousse',
 'churros',
 'clam chowder',
 'club sandwich',
 'crab cakes',
 'creme brulee',
 'croque madame',
 'cup cakes',
 'deviled eggs',
 'donuts',
 'dumplings',
 'edamame',
 'eggs benedict',
 'escargots',
 'falafel',
 'filet mignon',
 'fish and chips',
 'foie gras',
 'french fries',
 'french onion soup',
 'french toast',
 'fried calamari',
 'fried rice',
 'frozen yogurt',
 'garlic bread',
 'gnocchi',
 'greek salad',
 'grilled cheese sandwich',
 'grilled salmon',
 'guacamole',
 'gyoza',
 'hamburger',
 'hot and sour soup',
 'hot dog',
 'huevos rancheros',
 'hummus',
 'ice cream',
 'lasagna',
 'lobster bisque',
 'lobster

In [26]:
food_nocaption = food[food.caption == '']
food_nocaption.shape

(52821, 4)

In [27]:
### save uncaptioned food item list
food_nocaption_list = ' '.join([c + '.jpg' for c in food_nocaption.photo_id])
f = open('food_nocaption_list.txt','w')
f.write(food_nocaption_list)
f.close

In [30]:
food_food101caption = food[food.caption.isin(label_list)]
food_food101caption.shape

(1429, 4)

In [31]:
### save food items captioned with food101 labels
food_food101caption_list = ' '.join([c + '.jpg' for c in food_food101caption.photo_id])
f = open('food_food101caption_list.txt','w')
f.write(food_food101caption_list)
f.close()

<function TextIOWrapper.close()>

In [35]:
label_words = pd.Series(' '.join(label_list).split(' ')).unique()
len(label_words)

139

In [40]:
words_re = re.compile("|".join(label_words))
food_not101 = food[[c != '' and (not bool(words_re.search(c))) for c in food.caption]]
food_not101.shape

(15879, 4)

In [46]:
food_not101.caption[0:5]

26    this was the shareable drink called one big sw...
39                                               dinner
41                        what i thought i was getting!
43    this is what the waffle was supposed to look l...
91                                         grbic cevapi
Name: caption, dtype: object

In [47]:
### save food items captioned without food101 label words
food_no_101label_words_list = ' '.join([c + '.jpg' for c in food_not101.photo_id])
f = open('food_no_101label_words_list.txt','w')
f.write(food_no_101label_words_list)
f.close()